In [2]:
with open ("test2.csv", "r") as t1: ###Open the document created from dataframe workflow
    dict_SNPs={} ###Calculate major, minor, and wildtypes by going through each line
    for lines in t1: ##Add number of major, minor, 
        currentVAF=0
        if "Sequence Name" not in lines: 
            if "->" in lines.split(",")[7]: ###Check if there are arrows and change of VAF due to difference of reads
                currentVAF=float(lines.split(",")[7][0:4].strip("%")) ###Assign the left side of the VAF when there is arrow
            else:
                currentVAF=float(lines.split(",")[7].strip("%")) ###If there is no arrow just strip percentage and assign VAF
            if currentVAF >=50: ##If VAF is larger or same as 50 then VAF is assigned as major
                #print(lines.split(","))
                if (lines.split(",")[1],"Major",lines.split(",")[9]) not in dict_SNPs:
                    dict_SNPs[lines.split(",")[1],"Major",lines.split(",")[9]]=1
                else:
                    dict_SNPs[lines.split(",")[1],"Major",lines.split(",")[9]]=dict_SNPs[lines.split(",")[1],"Major",lines.split(",")[9]]+1
            if 0<currentVAF<50: ###If VAF is smaller than 50 but larger than 0 than VAF is assigned as minor
                if (lines.split(",")[1],"Minor",lines.split(",")[9]) not in dict_SNPs:
                    dict_SNPs[lines.split(",")[1],"Minor",lines.split(",")[9]]=1
                else:
                    dict_SNPs[lines.split(",")[1],"Minor",lines.split(",")[9]]=dict_SNPs[lines.split(",")[1],"Minor",lines.split(",")[9]]+1
            if currentVAF ==0: ## If VAF is 0 then it is assigned as wildtype
                if (lines.split(",")[1],"Wildtype",lines.split(",")[9]) not in dict_SNPs:
                    dict_SNPs[lines.split(",")[1],"Wildtype",lines.split(",")[9]]=1
                else:
                    dict_SNPs[lines.split(",")[1],"Wildtype",lines.split(",")[9]]=dict_SNPs[lines.split(",")[1],"Wildtype",lines.split(",")[9]]+1

with open ("test2.csv", "r") as t1: ##Make another dictionary for G_annotation without last letter of SNP changes so that we could assign wildtypes to same amino acid change
    dict_SNPs_minus={} ###Calculate major, minor, and wildtypes by going through each line. Subtract the last letter from the SNP to cover several mutations.
    for lines in t1: ###The process is similar to above for loop except the item is SNP without last letter
        currentVAF=0
        if "Sequence Name" not in lines:
            if "->" in lines.split(",")[7]:
                currentVAF=float(lines.split(",")[7][0:4].strip("%"))
            else:
                currentVAF=float(lines.split(",")[7].strip("%"))
            if currentVAF >=50:
                if (lines.split(",")[1],"Major",lines.split(",")[9][0:-1]) not in dict_SNPs_minus:
                    dict_SNPs_minus[lines.split(",")[1],"Major",lines.split(",")[9][0:-1]]=1
                else:
                    dict_SNPs_minus[lines.split(",")[1],"Major",lines.split(",")[9][0:-1]]=dict_SNPs_minus[lines.split(",")[1],"Major",lines.split(",")[9][0:-1]]+1
            if 0<currentVAF<50:
                if (lines.split(",")[1],"Minor",lines.split(",")[9][0:-1]) not in dict_SNPs_minus:
                    dict_SNPs_minus[lines.split(",")[1],"Minor",lines.split(",")[9][0:-1]]=1
                else:
                    dict_SNPs_minus[lines.split(",")[1],"Minor",lines.split(",")[9][0:-1]]=dict_SNPs_minus[lines.split(",")[1],"Minor",lines.split(",")[9][0:-1]]+1
            if currentVAF ==0:
                if (lines.split(",")[1],"Wildtype",lines.split(",")[9][0:-1]) not in dict_SNPs_minus:
                    dict_SNPs_minus[lines.split(",")[1],"Wildtype",lines.split(",")[9][0:-1]]=1
                else:
                    dict_SNPs_minus[lines.split(",")[1],"Wildtype",lines.split(",")[9][0:-1]]=dict_SNPs_minus[lines.split(",")[1],"Wildtype",lines.split(",")[9][0:-1]]+1

dict_SNPs_SUM={} ###Find sum of major, minor, wildtypes, total  samples
dict_minus_test={} ###Make a dictionary for minus 


dict_SNPs_2={} ##The purpose of adding wildtype to second mutations
for items in dict_SNPs:
    if (items[0], "Wildtype", items[2]) not in dict_SNPs: ##Check if wildtype is not in the dictionary 
        dict_SNPs_2[items[0], "Wildtype", items[2]]=dict_SNPs_minus[items[0],"Wildtype",items[2][0:-1]] ###Assign wildtype to the SNP change without wildtype information

for items in dict_SNPs_2: ##Add back to the original dictionary
    dict_SNPs[items]=dict_SNPs_2[items]
    

for items in dict_SNPs: ##Create a beginng value of dictionary to later add up to total
    dict_SNPs_SUM[items[0],items[2]]=0
    
###Figure out why sum  is weird for seaborn table why is it differet from WVAF total calculation so much?

dict_count={} ## dictionary for couting how many times dict_SNP_minus which is without last letter of G_Annotation is added.
for items in dict_SNPs: ##First check if the item is in dictionary otherwise change the value of dict_count
    if (items[0],items[2]) in dict_count and items[1]=="Wildtype":
        dict_count[items[0],items[2]]=1
    if (items[0],items[2]) not in dict_count and items[1]=="Wildtype":
        dict_count[items[0],items[2]]=0
    ##Only add the dict_SNP which is SNP value for wildtype once because there would be only one wildtype value
    if items[1]=="Wildtype":
        if dict_count[items[0],items[2]]<1:
            dict_SNPs_SUM[items[0],items[2]]=dict_SNPs_SUM[items[0],items[2]]+dict_SNPs[items]
    if items[1]!="Wildtype":
        dict_SNPs_SUM[items[0],items[2]]=dict_SNPs_SUM[items[0],items[2]]+dict_SNPs[items]

        
dict_SNPS_ratio = {} ##Find the ratio of major, minor, wildtypes in total samples
for items in dict_SNPs:
    dict_SNPS_ratio[items]=round(dict_SNPs[items]/dict_SNPs_SUM[items[0],items[2]],5)


dict_SNPs_final = {} ##Create final form for the SNPs
for items in dict_SNPs:
    if (items[0],items[2]) not in dict_SNPs_final:
        dict_SNPs_final[items[0],items[2]]=[0,0,0,0]
    if items[1]=="Wildtype":
        dict_SNPs_final[items[0],items[2]][0]=dict_SNPS_ratio[items]
    if items[1]=="Major":
        dict_SNPs_final[items[0],items[2]][1]=dict_SNPS_ratio[items]
    if items[1]=="Minor":
        dict_SNPs_final[items[0],items[2]][2]=dict_SNPS_ratio[items]
    dict_SNPs_final[items[0],items[2]][3]=dict_SNPs_SUM[items[0],items[2]]

with open ("Tab_Table_SNP.csv", "w") as t2:
    t2.write("Gene+SNP,Wildtype: AF=0%,Minor: AF < 50%,Major: AF >= 50%,Samplesize\n")
    for items in dict_SNPs_final:
        if items[0]=="DHPS_437Corrected ":
            t2.write("DHPS"+":"+items[1]+","+str(dict_SNPs_final[items][0])+","+str(dict_SNPs_final[items][1])+","+str(dict_SNPs_final[items][2])+","+"N="+str(dict_SNPs_final[items][3])+"\n")
        if items[0]=="mitochondrial genome - CYTB CDS":
            t2.write("CYTB"+":"+items[1]+","+str(dict_SNPs_final[items][0])+","+str(dict_SNPs_final[items][1])+","+str(dict_SNPs_final[items][2])+","+"N="+str(dict_SNPs_final[items][3])+"\n")         
        if items[0]!= "DHPS_437Corrected " and items[0]!="mitochondrial genome - CYTB CDS":
            t2.write(items[0]+":"+items[1]+","+str(dict_SNPs_final[items][0])+","+str(dict_SNPs_final[items][1])+","+str(dict_SNPs_final[items][2])+","+"N="+str(dict_SNPs_final[items][3])+"\n")
    



['19ANBe00xp001PfF1152', 'pooled', '19', 'Benguela', '0', 'DHFR', 'S108N', '325', '100.0%', '325', 'mutation', '108N', '1.0', '3.0', '300.0\n']
['19ANBe00xp003PfF1152', 'pooled', '19', 'Benguela', '0', 'DHFR', 'S108N', '1594', '100.0%', '1594', 'mutation', '108N', '3.0', '10.0', '1000.0\n']
['19ANBe00xp004PfF1152', 'pooled', '19', 'Benguela', '0', 'DHFR', 'S108N', '1200', '100.0%', '1200', 'mutation', '108N', '4.0', '10.0', '1000.0\n']
['19ANBe00xp005PfF1152', 'pooled', '19', 'Benguela', '0', 'DHFR', 'S108N', '1629', '100.0%', '1629', 'mutation', '108N', '5.0', '10.0', '1000.0\n']
['19ANBe00xp006PfF1152', 'pooled', '19', 'Benguela', '0', 'DHFR', 'S108N', '1216', '99.7%', '1212', 'mutation', '108N', '6.0', '10.0', '997.0\n']
['19ANBe00xp007PfF1152', 'pooled', '19', 'Benguela', '0', 'DHFR', 'S108N', '2103', '100.0%', '2102', 'mutation', '108N', '7.0', '10.0', '1000.0\n']
['19ANBe00xp008PfF1152', 'pooled', '19', 'Benguela', '0', 'DHFR', 'S108N', '986', '100.0%', '986', 'mutation', '108N',

ValueError: could not convert string to float: '10 -'